# Simple Demo - Reviewing WEC Laptime Data

For many forms of motorsport, timing data in the form of laptime data is often made available at the end of the race. This data can be used by fans and sports data journalists alike, as well as teams and drivers, for getting a *post hoc* insight into what actuall went on in a race.

*Live laptime data is also typically provided via live timing screens, from which data feeds may be available to teams, although not, typically, to the public. (Data from timing screens can still be scraped unofficially, though...) Having access to laptime history during a race can be useful for reviewing how a race is evolving, plotting strategy, and even predicting likely future race events, such as overtaking possibilities or pit stops due to degrading laptimes.*

This notebook provides a basic demonstration of how to download and analyse [FIA World Endurance Championship (WEC) data from Al Kamel Systems](http://fiawec.alkamelsystems.com), getting a feel for what the data can tell us and how we might encourage it to reveal some of the stories it undoubtedly contains.

Timing info available as PDF, eg [here](https://assets.lemans.org/explorer/pdf/courses/2019/24-heures-du-mans/classification/race/24-heures-du-mans-2019-classification-after-24h.pdf).

CSV data from Al Kamel using links of form:

`http://fiawec.alkamelsystems.com/Results/08_2018-2019/07_SPA%20FRANCORCHAMPS/267_FIA%20WEC/201905041330_Race/Hour%206/23_Analysis_Race_Hour%206.CSV`


*Links don't seem to appear on e.g. [classification data](http://fiawec.alkamelsystems.com/)? So where else might they be found?*

In [1]:
#Enable inline plots
%matplotlib inline

# pandas is a python package for working with tabular datasets
import pandas as pd

In [5]:
#Add the parent dir to the import path
import sys
sys.path.append("../py")

#Import contents of the utils.py package in the ../py directory
from utils import *

In [8]:
#Download URL for timing data in CSV format
url = 'http://fiawec.alkamelsystems.com/Results/08_2018-2019/07_SPA%20FRANCORCHAMPS/267_FIA%20WEC/201905041330_Race/Hour%206/23_Analysis_Race_Hour%206.CSV'


Download the data into a *pandas* dataframe and preview the first few rows:

In [ ]:
laptimes = pd.read_csv(url, sep=';').dropna(how='all', axis=1)
laptimes.head()

We can save the raw data to a local CSV file so we can access it locally at a future time:

In [ ]:
## TO DO - unescape the %20... 
outfile_name = url.split('/')[-1])
laptimes.to_csv('../data/{}'.format(outfile_name)

In [17]:
#Load data from local file
#laptimes = pd.read_csv('23_Analysis_Race_Hour 6.csv', sep=';').dropna(how='all', axis=1)
#laptimes.head()

In [19]:
#Clean the column names of any leading / trailing whitespace
laptimes.columns = [c.strip() for c in laptimes.columns]

#Tidy the data a little... car and driver number are not numbers
laptimes[['NUMBER','DRIVER_NUMBER']] = laptimes[['NUMBER','DRIVER_NUMBER']].astype(str)

laptimes.head()

,NUMBER,DRIVER_NUMBER,LAP_NUMBER,LAP_TIME,LAP_IMPROVEMENT,CROSSING_FINISH_LINE_IN_PIT,S1,S1_IMPROVEMENT,S2,S2_IMPROVEMENT,...,S1_LARGE,S2_LARGE,S3_LARGE,TOP_SPEED,DRIVER_NAME,PIT_TIME,CLASS,GROUP,TEAM,MANUFACTURER
0,1,1,1,2:06.349,0,NaN,37.900,0,57.533,0,...,0:37.900,0:57.533,0:30.916,289.0,Neel JANI,NaN,LMP1,NaN,Rebellion Racing,Rebellion
1,1,1,2,2:00.417,0,NaN,34.830,0,54.824,0,...,0:34.830,0:54.824,0:30.763,302.7,Neel JANI,NaN,LMP1,NaN,Rebellion Racing,Rebellion
2,1,1,3,2:17.278,0,B,40.012,0,55.696,0,...,0:40.012,0:55.696,0:41.570,289.0,Neel JANI,NaN,LMP1,NaN,Rebellion Racing,Rebellion
3,1,1,4,3:10.094,0,NaN,1:26.938,0,1:07.045,0,...,1:26.938,1:07.045,0:36.111,262.4,Neel JANI,0:00:58.509,LMP1,NaN,Rebellion Racing,Rebellion
4,1,1,5,2:29.763,0,NaN,41.205,0,1:11.517,0,...,0:41.205,1:11.517,0:37.041,259.2,Neel JANI,NaN,LMP1,NaN,Rebellion Racing,Rebellion


In [20]:
# Review the column headings
laptimes.columns

Index(['NUMBER', 'DRIVER_NUMBER', 'LAP_NUMBER', 'LAP_TIME', 'LAP_IMPROVEMENT',
       'CROSSING_FINISH_LINE_IN_PIT', 'S1', 'S1_IMPROVEMENT', 'S2',
       'S2_IMPROVEMENT', 'S3', 'S3_IMPROVEMENT', 'KPH', 'ELAPSED', 'HOUR',
       'S1_LARGE', 'S2_LARGE', 'S3_LARGE', 'TOP_SPEED', 'DRIVER_NAME',
       'PIT_TIME', 'CLASS', 'GROUP', 'TEAM', 'MANUFACTURER'],
      dtype='object')

The `DRIVER_NUMBER` is relative to a car. It may be useful to also have a unique driver number, `CAR_DRIVER`:

In [21]:
laptimes['CAR_DRIVER'] = laptimes['NUMBER'] + '_' + laptimes['DRIVER_NUMBER']
laptimes[['NUMBER','DRIVER_NUMBER','CAR_DRIVER']].head()

,NUMBER,DRIVER_NUMBER,CAR_DRIVER
0,1,1,1_1
1,1,1,1_1
2,1,1,1_1
3,1,1,1_1
4,1,1,1_1


## Quick demo chart

Some simple plots to show how we can use widgets etc.

In [22]:
laptimes['LAP_TIME_S'] = laptimes['LAP_TIME'].apply(getTime)
laptimes[['LAP_TIME','LAP_TIME_S']].head()

,LAP_TIME,LAP_TIME_S
0,2:06.349,126.349
1,2:00.417,120.417
2,2:17.278,137.278
3,3:10.094,190.094
4,2:29.763,149.763


Let's start with a plot that allows us to select a particular car number and plot the laptimes associated with it:

In [23]:
from ipywidgets import interact

@interact(number=laptimes['NUMBER'].unique().tolist(),)
def plotLapByNumber(number):
    laptimes[laptimes['NUMBER']==number].plot(x='LAP_NUMBER',y='LAP_TIME_S')

interactive(children=(Dropdown(description='number', options=('1', '11', '17', '26', '28', '29', '3', '31', '3…

We can also highlight which laps were driven by which driver by splitting the data for a particular car out over several columns, one for each driver, and then potting each driver column using a separate colour.

The table is reshaped using the *pandas* `pivot()` method, setting the lap number as the *index* of the pivoted dataframe and splitting the `LAP_TIME_S` *values* out over several new *columns* that identify each `DRIVER_NUMBER`:

In [27]:
laptimes[laptimes['NUMBER']=='1'].pivot(index='LAP_NUMBER',
                                        columns='DRIVER_NUMBER',
                                        values='LAP_TIME_S').head()

DRIVER_NUMBER,1,2,3
LAP_NUMBER,,,
1,126.349,NaN,NaN
2,120.417,NaN,NaN
3,137.278,NaN,NaN
4,190.094,NaN,NaN
5,149.763,NaN,NaN


The `.plot()` command will, by default, plot the values in each column of a dataframe as a separate line against the corresponding index values:

In [24]:
@interact(number=laptimes['NUMBER'].unique().tolist(),)
def plotLapByNumberDriver(number):
    # We can pivot long to wide on driver number,
    # then plot all cols against the lapnumber index
    laptimes[laptimes['NUMBER']==number].pivot(index='LAP_NUMBER',
                                               columns='DRIVER_NUMBER',
                                               values='LAP_TIME_S').plot()


interactive(children=(Dropdown(description='number', options=('1', '11', '17', '26', '28', '29', '3', '31', '3…

We can also add annotations to the chart. For example, we might want to identify laps on which the car pitted so that we can disambiguate slow laps caused by an on-track incident, for example, from laps where the driver went through the pit lane.

*(Depending on the timing marker, lap times for laps where the car crossed the finish line in the pit may or may not include the pit stop time.)*

In [32]:
@interact(number=laptimes['NUMBER'].unique().tolist(),)
def plotLapByNumberDriverWithPit(number):
    # We can pivot long to wide on driver number,
    # then plot all cols against the lapnumber index
    #Grap the matplotli axes so we can overplot onto them
    ax = laptimes[laptimes['NUMBER']==number].pivot(index='LAP_NUMBER',
                                                    columns='DRIVER_NUMBER',
                                                    values='LAP_TIME_S').plot()
    # Also add in pit laps
    # Filter rows that identify both the car
    # and the laps on which the car crossed the finish line in the pit
    inpitlaps = laptimes[(laptimes['NUMBER']==number) & (laptimes['CROSSING_FINISH_LINE_IN_PIT']=='B')]
    # Plot a marker for each of those rows
    inpitlaps.plot.scatter(x='LAP_NUMBER',y='LAP_TIME_S', ax=ax)
    

interactive(children=(Dropdown(description='number', options=('1', '11', '17', '26', '28', '29', '3', '31', '3…

## Stint Detection

Looking at the chart of laptimes vs driver number, we see that each car is on track for several distinct contiguous lap periods, which we might describe as "stints".

We can identify several simple heuristics for identifying different sorts of stint:

- *car stint*: laps covered between each pit stop;
- *driver session*: session equates to continuous period in car;
- *driver stint*: relative to pit stops; this may be renumbered for each session?

In [ ]:
#Driver session

#Create a flag to identify when we enter the pit, aka an INLAP
laptimes['INLAP'] = laptimes['CROSSING_FINISH_LINE_IN_PIT'] == 'B'

#Make no assumptions about table order - so sort by lap number
laptimes = laptimes.sort_values(['NUMBER','LAP_NUMBER'])

# Identify a new stint for each car by sifting the pitting / INLAP flag within car tables
laptimes['OUTLAP'] = laptimes.groupby('NUMBER')['INLAP'].shift(fill_value=True)

laptimes[['DRIVER_NUMBER', 'INLAP','OUTLAP']].head()

In [ ]:
#This is a count of the number of times a driver is in a vehicle after a pit who wasn't in it before
#Also set overall lap = 1 to be a driver change
laptimes['driverchange'] = (~laptimes['DRIVER_NUMBER'].eq(laptimes['DRIVER_NUMBER'].shift())) | (laptimes['LAP_NUMBER']==1)

laptimes['DRIVER_SESSION'] = laptimes.groupby(['NUMBER', 'DRIVER_NUMBER'])['driverchange'].cumsum().astype(int)
laptimes[['DRIVER_NUMBER', 'driverchange','DRIVER_SESSION','LAP_NUMBER']][42:48]

In [ ]:
# Car stint
#Create a counter for each pit stop - the pit flag is entering pit at end of stint
#  so a new stint applies on the lap after a pit
#Find the car stint based on count of pit stops
laptimes['CAR_STINT'] = laptimes.groupby('NUMBER')['OUTLAP'].cumsum().astype(int)

laptimes[['CROSSING_FINISH_LINE_IN_PIT', 'INLAP', 'OUTLAP', 'CAR_STINT']].head()

In [ ]:
#Driver stint - a cumulative count for each driver of their stints
laptimes['DRIVER_STINT'] = laptimes.groupby('CAR_DRIVER')['OUTLAP'].cumsum().astype(int)

#Let's also derive another identifier - CAR_DRIVER_STINT
laptimes['CAR_DRIVER_STINT'] = laptimes['CAR_DRIVER'] + '_' + laptimes['DRIVER_STINT'].astype(str)

laptimes[['CAR_DRIVER', 'CROSSING_FINISH_LINE_IN_PIT', 'INLAP','CAR_STINT', 'DRIVER_STINT', 'CAR_DRIVER_STINT']].tail(20).head(10)


In [ ]:
#Driver session stint - a count for each driver of their stints within a particular driving session
laptimes['DRIVER_SESSION_STINT'] = laptimes.groupby(['CAR_DRIVER','DRIVER_SESSION'])['OUTLAP'].cumsum().astype(int)
laptimes[['CAR_DRIVER', 'CROSSING_FINISH_LINE_IN_PIT', 'INLAP','CAR_STINT', 'DRIVER_STINT', 'CAR_DRIVER_STINT', 'DRIVER_SESSION_STINT']].head()


## Lap Counts Within Stints

It may be convenient to keep track of lap counts within stints.

In [ ]:
# lap count by car stint - that is, between each pit stop
laptimes['LAPS_CAR_STINT'] = laptimes.groupby(['NUMBER','CAR_STINT']).cumcount()+1

#lap count by driver
laptimes['LAPS_DRIVER'] = laptimes.groupby('CAR_DRIVER').cumcount()+1

#lap count by driver session
laptimes['LAPS_DRIVER_SESSION'] = laptimes.groupby(['CAR_DRIVER','DRIVER_SESSION']).cumcount()+1

#lap count by driver stint
laptimes['LAPS_DRIVER_STINT'] = laptimes.groupby(['CAR_DRIVER','DRIVER_STINT']).cumcount()+1

laptimes[['LAPS_CAR_STINT', 'LAPS_DRIVER', 'LAPS_DRIVER_SESSION', 'LAPS_DRIVER_STINT']].tail()

## Basic Individal Driver Reports

Using those additional columns, we should be able to start creating reports by driver by facetting on individual drivers.

(Note: it might be interesting to do some datasette demos with particular facets, which make it easy to select teams, drivers, etc.)

In [ ]:
import qgrid
qgrid.show_grid(laptimes[['LAP_NUMBER', 'NUMBER', 'CAR_DRIVER',  'INLAP', 'CAR_STINT', 
                          'CAR_DRIVER_STINT', 'DRIVER_STINT', 'DRIVER_SESSION', 'DRIVER_SESSION_STINT']])

## Simple Stint Reports

Using the various stint details, we can pull together a simple set of widgets to allow us to explore times by car / driver.

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact

In [ ]:
cars = widgets.Dropdown(
    options=laptimes['NUMBER'].unique(), # value='1',
    description='Car:', disabled=False )

drivers = widgets.Dropdown(
    options=laptimes[laptimes['NUMBER']==cars.value]['CAR_DRIVER'].unique(),
    description='Driver:', disabled=False)

driversessions = widgets.Dropdown(
    options=laptimes[laptimes['CAR_DRIVER']==drivers.value]['DRIVER_SESSION'].unique(),
    description='Session:', disabled=False)

driverstints = widgets.Dropdown(
    options=laptimes[laptimes['DRIVER_SESSION']==driversessions.value]['DRIVER_SESSION_STINT'].unique(),
    description='Stint:', disabled=False)

def update_drivers(*args):
    driverlist = laptimes[laptimes['NUMBER']==cars.value]['CAR_DRIVER'].unique()
    drivers.options = driverlist
    
def update_driver_session(*args):
    driversessionlist = laptimes[(laptimes['CAR_DRIVER']==drivers.value)]['DRIVER_SESSION'].unique()
    driversessions.options = driversessionlist
    
def update_driver_stint(*args):
    driverstintlist = laptimes[(laptimes['CAR_DRIVER']==drivers.value) &
                               (laptimes['DRIVER_SESSION']==driversessions.value)]['DRIVER_SESSION_STINT'].unique()
    driverstints.options = driverstintlist
    
cars.observe(update_drivers, 'value')
drivers.observe(update_driver_session,'value')
driversessions.observe(update_driver_stint,'value')

def laptime_table(car, driver, driversession, driverstint):
    #just basic for now...
    display(laptimes[(laptimes['CAR_DRIVER']==driver) &
                     (laptimes['DRIVER_SESSION']==driversession) &
                     (laptimes['DRIVER_SESSION_STINT']==driverstint) ][['CAR_DRIVER', 'DRIVER_SESSION',
                                                         'DRIVER_STINT', 'DRIVER_SESSION_STINT',
                                                         'LAP_NUMBER','LAP_TIME', 'LAP_TIME_S']])
    
interact(laptime_table, car=cars, driver=drivers, driversession=driversessions, driverstint=driverstints);


In [ ]:
def laptime_chart(car, driver, driversession, driverstint):
    tmp_df = laptimes[(laptimes['CAR_DRIVER']==driver) &
                     (laptimes['DRIVER_SESSION']==driversession) &
                     (laptimes['DRIVER_SESSION_STINT']==driverstint) ][['CAR_DRIVER', 'DRIVER_SESSION',
                                                         'DRIVER_STINT', 'DRIVER_SESSION_STINT',
                                                         'LAP_NUMBER','LAP_TIME', 'LAP_TIME_S']]['LAP_TIME_S'].reset_index(drop=True)
    if not tmp_df.empty:
        tmp_df.plot()
        
interact(laptime_chart, car=cars, driver=drivers, driversession=driversessions, driverstint=driverstints);


In [ ]:
#Also add check boxes to suppress inlap and outlap?
inlaps = widgets.Checkbox( value=True, description='Inlap',
                           disabled=False )

outlaps = widgets.Checkbox( value=True, description='Outlap',
                           disabled=False )


#Plot laptimes by stint for a specified driver
def laptime_charts(car, driver, driversession, inlap, outlap):
    tmp_df = laptimes
    
    if not inlap:
        tmp_df = tmp_df[~tmp_df['INLAP']]
    if not outlap:
        tmp_df = tmp_df[~tmp_df['OUTLAP']]
        
    tmp_df = tmp_df[(tmp_df['CAR_DRIVER']==driver) &
                     (tmp_df['DRIVER_SESSION']==driversession) ].pivot(index='LAPS_DRIVER_STINT',
                                                                       columns='DRIVER_SESSION_STINT', 
                                                                       values='LAP_TIME_S').reset_index(drop=True)
    
    if not tmp_df.empty:
        tmp_df.plot()



interact(laptime_charts, car=cars, driver=drivers, driversession=driversessions, inlap=inlaps, outlap=outlaps);



## Simple Model

An example of creating a simple model using some explcitly pulled out data.

In [ ]:
def sample_laptimes(df, driver,driversession,driversessionstint=None, inlap=False, outlap=False):
    df = df[(df['CAR_DRIVER']==driver) & (df['DRIVER_SESSION']== driversession)]
    if not inlap:
        df = df[~df['INLAP']]
    if not outlap:
        df = df[~df['OUTLAP']]
        
    if driversessionstint:
        return df[df['DRIVER_SESSION_STINT']==driversessionstint]['LAP_TIME_S']
    
    return df.pivot(index='LAPS_DRIVER_STINT',
                    columns='DRIVER_SESSION_STINT',
                    values='LAP_TIME_S').reset_index(drop=True)

sample_laptimes(laptimes,'56_1', 1, 2)

### Some Simple Linear Models

Fitted without explanation... we should only accept models that appear to fit, i.e. that have a low error...

In [ ]:
import seaborn as sns

sns.lmplot(x='index', y='LAP_TIME_S', data=sample_laptimes(laptimes,'56_1', 1, 2).reset_index());


We can also increase the order of the fit line (the `ci` parameter toggles the confidence bounds display):

In [ ]:
sns.lmplot(x='index', y='LAP_TIME_S', data=sample_laptimes(laptimes,'56_1', 1, 2).reset_index(),
           order = 2, ci=None);


In [ ]:
#Simple model
import statsmodels.api as sm


Y = sample_laptimes(laptimes,'56_1', 1, 2).reset_index(drop=True)
X = Y.index.values
X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

In [ ]:
p = model.params

ax = pd.DataFrame(Y).reset_index().plot(kind='scatter', x='index', y='LAP_TIME_S')
ax.plot(X, p.const + p.x1 * X);

In [ ]:
import matplotlib.pyplot as plt

# scatter-plot data
fig, ax = plt.subplots()
fig = sm.graphics.plot_fit(model, 0, ax=ax)

## Simple Position Calculations

Some simple demonstrations of calculating position data.

Naively, calculate position based on lap number and accumulated time (there may be complications based on whether the lead car records a laptine from pit entry...).

In [ ]:
#Find accumulated time in seconds
laptimes['ELAPSED_S']=laptimes['ELAPSED'].apply(getTime)


#Check
laptimes['CHECK_ELAPSED_S'] = laptimes.groupby('NUMBER')['LAP_TIME_S'].cumsum()

laptimes[['ELAPSED','ELAPSED_S','CHECK_ELAPSED_S']].tail()

We can use the position to identify the leader on each lap and from that a count of leadlap number for each car:

In [ ]:
#Find position based on accumulated laptime
laptimes = laptimes.sort_values('ELAPSED_S')
laptimes['POS'] = laptimes.groupby('LAP_NUMBER')['ELAPSED_S'].rank()

#Find leader naively
laptimes['leader'] = laptimes['POS']==1

#Find lead lap number
laptimes['LEAD_LAP_NUMBER'] = laptimes['leader'].cumsum()

laptimes[['LAP_NUMBER','LEAD_LAP_NUMBER']].tail()

## Simple Position Chart - Top 10 At End

Find last lap number, then get top 10 on that lap.

In [ ]:
LAST_LAP = laptimes['LEAD_LAP_NUMBER'].max()
LAST_LAP

In [ ]:
#Find top 10 at end
cols = ['NUMBER','TEAM', 'DRIVER_NAME', 'CLASS','LAP_NUMBER','ELAPSED']
top10 = laptimes[laptimes['LEAD_LAP_NUMBER']==LAST_LAP].sort_values(['LEAD_LAP_NUMBER', 'POS'])[cols].head(10).reset_index(drop=True)
top10.index += 1
top10